In [17]:
import pandas.io.sql as sqlio
import psycopg2 
import requests, json
import subprocess

### # Docker command to start a database

In [18]:
subprocess.run(['docker', 'run', '-d', '--name=starshipsdb', '-e', 'POSTGRES_USER=altagram', '-e', 'POSTGRES_PASSWORD=altagram21', '-p', '127.0.0.1:5432:5432', 'postgres:13'])

CompletedProcess(args=['docker', 'run', '-d', '--name=starshipsdb', '-e', 'POSTGRES_USER=altagram', '-e', 'POSTGRES_PASSWORD=altagram21', '-p', '127.0.0.1:5432:5432', 'postgres:13'], returncode=0)

### # Retrieve starship data from swapi

In [19]:
starships = []
url = "https://swapi.dev/api/starships/"
while url != None:
    response = requests.get(url, verify=False)
    json = response.json()
    starships.extend(json['results'])
    url = json['next']

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:849: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:849: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:849: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:849: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate ver

### # Connect to the database

In [21]:
connection = psycopg2.connect(host='localhost', user='altagram', password='altagram21')
cursor = connection.cursor()

### # Create a table and insert data

In [22]:
sql_create_table = """CREATE TABLE IF NOT EXISTS starships (
        ship_name TEXT NOT NULL PRIMARY KEY,
        rating DECIMAL)"""

cursor.execute(sql_create_table)

In [23]:
sql_insert = """INSERT INTO starships(ship_name, rating)
             VALUES(%s, %s) ON CONFLICT DO NOTHING"""

for starship in starships:
    if starship['hyperdrive_rating'] != "unknown":
        cursor.execute(sql_insert, (starship['name'],starship['hyperdrive_rating']))

### # Request the data from the table and display with pandas

In [24]:
sqlio.read_sql_query("SELECT * FROM starships ORDER BY rating ASC", connection)

,ship_name,rating
0,Naboo star skiff,0.5
1,Millennium Falcon,0.5
2,Republic Assault ship,0.6
3,J-type diplomatic barge,0.7
4,H-type Nubian yacht,0.9
5,Republic attack cruiser,1.0
6,Jedi Interceptor,1.0
7,TIE Advanced x1,1.0
8,Sentinel-class landing craft,1.0
9,Banking clan frigte,1.0


In [ ]:
cursor.close()
connection.commit()
connection.close()

### # Shutdown the postgres database

In [ ]:
subprocess.run(['docker', 'rm', '-f', 'starshipsdb'])